In [1]:
import pandas as pd

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report



In [3]:
df=pd.read_csv("train_with_labels.csv")

In [4]:
df['create_dt'] = pd.to_datetime(df['create_dt'], format='%Y-%m-%d %H:%M:%S')

In [5]:
df['create_dt_hour'] = df['create_dt'].dt.hour
df['create_dt_minute'] = df['create_dt'].dt.minute
df['create_dt_second'] = df['create_dt'].dt.second

In [6]:
df['create_dt_hms'] = df['create_dt'].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)

In [7]:
df['create_dt_hms'] = df['create_dt_hms'].astype('float64')

In [8]:
df['create_dt_hms'] = pd.to_numeric(df['create_dt_hms'])

In [10]:
df.drop(['create_dt'], axis=1, inplace=True)
df.drop(['create_dt_hour'], axis=1, inplace=True)
df.drop(['create_dt_minute'], axis=1, inplace=True)
df.drop(['create_dt_second'], axis=1, inplace=True)
# df.drop(['name_idle'], axis=1, inplace=True)
# df.drop(['name_loading'], axis=1, inplace=True)
# df.drop(['name_riding_empty'], axis=1, inplace=True)
# df.drop(['name_riding_loaded'], axis=1, inplace=True)
# df.drop(['name_unloading_lift'], axis=1, inplace=True)
df.drop(['name'], axis=1, inplace=True)
# df.drop(['format'], axis=1, inplace=True)

In [11]:
df.drop(['format'], axis=1, inplace=True)

In [12]:
print(df.dtypes)

mdm_object_name        int64
mdm_model_id           int64
lon                  float64
lat                  float64
alt                    int64
speed_gps            float64
direction            float64
accel_forward_nn     float64
accel_braking_nn     float64
accel_angular_nn     float64
accel_vertical_nn    float64
operation_kind_id    float64
create_dt_hms        float64
dtype: object


In [13]:
df.dropna(inplace=True)

In [23]:
df2=pd.read_csv("dataset/telemetry_for_operations_validation.csv")

In [25]:
# Check if the format is 'YYYY-MM-DD HH:MM:SS'
def check_format(date_str):
    # Try to convert to datetime with a specific format
    try:
        pd.to_datetime(date_str, format='%Y-%m-%d %H:%M:%S')
        return 'datetime'
    except ValueError:
        try:
            pd.to_datetime(date_str, format='%Y-%m-%d')
            return 'date'
        except ValueError:
            return 'invalid'

# Apply the function to the 'create_dt' column
df2['format'] = df2['create_dt'].apply(check_format)

In [ ]:
# Display the rows with 'date' or 'invalid' format
df2=df2.drop(df2[df2['format'] != 'datetime'].index)


In [26]:
df2['create_dt'] = pd.to_datetime(df2['create_dt'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

In [27]:
df2['create_dt_hour'] = df2['create_dt'].dt.hour
df2['create_dt_minute'] = df2['create_dt'].dt.minute
df2['create_dt_second'] = df2['create_dt'].dt.second

In [28]:
df2['create_dt_hms'] = df2['create_dt'].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)

In [29]:
df2['create_dt_hms'] = df2['create_dt_hms'].astype('float64')

In [30]:
df2['create_dt_hms'] = pd.to_numeric(df2['create_dt_hms'])

In [32]:
 df2.drop(['create_dt'], axis=1, inplace=True)
df2.drop(['create_dt_hour'], axis=1, inplace=True)
df2.drop(['create_dt_minute'], axis=1, inplace=True)
df2.drop(['create_dt_second'], axis=1, inplace=True)
# df2.drop(['name_idle'], axis=1, inplace=True)
# df2.drop(['name_loading'], axis=1, inplace=True)
# df2.drop(['name_riding_empty'], axis=1, inplace=True)
# df2.drop(['name_riding_loaded'], axis=1, inplace=True)
# df2.drop(['name_unloading_lift'], axis=1, inplace=True)
# df2.drop(['name'], axis=1, inplace=True)
df2.drop(['format'], axis=1, inplace=True)

In [33]:
print(df2.dtypes)

mdm_object_name        int64
mdm_model_id           int64
lon                  float64
lat                  float64
alt                    int64
speed_gps            float64
direction            float64
accel_forward_nn     float64
accel_braking_nn     float64
accel_angular_nn     float64
accel_vertical_nn    float64
create_dt_hms        float64
dtype: object


In [26]:
df2.dropna(inplace=True)

In [34]:
X2 = df2.drop('operation_kind_id', axis=1)  # features
y2 = df2['operation_kind_id']  # target variable


KeyError: "['operation_kind_id'] not found in axis"

In [16]:
# split data into training and testing sets
X = df.drop('operation_kind_id', axis=1)  # features
y = df['operation_kind_id']  # target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create a Random Forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# train the model
rf.fit(X_train, y_train)

# make predictions on the test set
y_pred = rf.predict(X_test)

# evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.977479402498534
Classification Report:
              precision    recall  f1-score   support

         0.0       0.98      0.94      0.96     13770
         1.0       0.99      0.99      0.99      6241
         2.0       0.98      0.99      0.98     47050
         3.0       0.93      0.95      0.94      1033
         5.0       0.97      0.98      0.97     32525

    accuracy                           0.98    100619
   macro avg       0.97      0.97      0.97    100619
weighted avg       0.98      0.98      0.98    100619



In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [10, 50, 100, 200, 500],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10],
    'max_features': ['auto', 'sqrt', 'log2']
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X, y)

print("Best parameters: ", grid_search.best_params_)
print("Best F1 score: ", grid_search.best_score_)

In [17]:
from sklearn.metrics import f1_score

# y_true = df['operation_kind_id']
# y_pred = rf.predict(X_train)

print(f1_score(y_test, y_pred, average='weighted'))

0.9774336343853626


In [35]:
y2_pred = rf.predict(df2)


In [40]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(rf, X, y, cv=5, scoring='f1_macro')
print("Cross-validation F1 score: ", scores.mean())

Cross-validation F1 score:  0.4005503836080366


In [ ]:
X2 = df2.drop('operation_kind_id', axis=1)  # features
y2 = df3['operation_kind_id']  # target variable


In [ ]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(f1_score(y_test, y2_pred, average='weighted'))

In [36]:
print(y2_pred)

[5. 5. 5. ... 5. 5. 5.]


In [38]:
df3=pd.read_csv("dataset/telemetry_for_operations_validation.csv")

In [39]:
import pandas as pd

# Create a new DataFrame with the desired columns
uploading_df = pd.DataFrame({
    'create_dt': df3['create_dt'],
    'mdm_object_name': df3['mdm_object_name'],
    'operation_kind_id': y2_pred
})

# Save the DataFrame to a new CSV file
uploading_df.to_csv('uploading_df_with_pred.csv', index=False)